In [1]:
import scipy
import scipy.linalg
import scipy.sparse.linalg
import numpy as np
import matplotlib.pyplot as plt
import ed
%matplotlib inline

In [2]:
L = 14
g = 1.5
J = 1.
sx_list = ed.gen_sx_list(L)
sz_list = ed.gen_sz_list(L)
H = ed.gen_hamiltonian(sx_list, sz_list, g, J)
E, vecs = scipy.sparse.linalg.eigsh(H, which='SA')
psi0 = vecs[:, 0]
print(psi0)
print("E =", E[0])
print("norm = ", np.linalg.norm(psi0)) # close enough :D

[-8.14694335e-01  2.76165437e-16  3.41897971e-16 ...  1.17593800e-17
 -1.03993899e-17 -4.07427082e-06]
E = -23.222959434117367
norm =  1.0000000000000007


In [3]:
 def compress(psi, L, chimax):
    psi_aR = np.reshape(psi, (1, 2**L))
    Ms = []
    for n in range(1, L+1):
        chi_n, dim_R = psi_aR.shape
        assert dim_R == 2**(L-(n-1))
        psi_LR = np.reshape(psi_aR, (chi_n*2, dim_R//2))
        M_n, lambda_n, psi_tilde = scipy.linalg.svd(psi_LR, full_matrices=False, lapack_driver='gesvd')
        if len(lambda_n) > chimax:
            keep = np.argsort(lambda_n)[::-1][:chimax]
            M_n = M_n[:, keep]
            lambda_n = lambda_n[keep]
            psi_tilde = psi_tilde[keep, :]
        chi_np1 = len(lambda_n)
        M_n = np.reshape(M_n, (chi_n, 2, chi_np1))
        Ms.append(M_n)
        psi_aR = lambda_n[:, np.newaxis] * psi_tilde[:,:]
    assert psi_aR.shape == (1, 1)
    print("remaining in compress: ", psi_aR)
    return Ms        
        

In [4]:
psi0_MPS_ex = compress(psi0, L, 2**(L//2))
for i in range(len(psi0_MPS_ex)):
    print(psi0_MPS_ex[i].shape)
print("first M:")
print(psi0_MPS_ex[0])
print("second M:")
print(psi0_MPS_ex[1])

remaining in compress:  [[-1.]]
(1, 2, 2)
(2, 2, 4)
(4, 2, 8)
(8, 2, 16)
(16, 2, 32)
(32, 2, 64)
(64, 2, 128)
(128, 2, 64)
(64, 2, 32)
(32, 2, 16)
(16, 2, 8)
(8, 2, 4)
(4, 2, 2)
(2, 2, 1)
first M:
[[[-1.00000000e+00 -4.09449408e-17]
  [-4.09449408e-17  1.00000000e+00]]]
second M:
[[[-9.86368001e-01  7.84633117e-18  5.78701312e-16  1.64554450e-01]
  [-3.00518954e-17 -9.48349869e-01  3.17226302e-01 -1.05448936e-15]]

 [[-5.63851524e-18  3.17226302e-01  9.48349869e-01 -3.33626869e-15]
  [ 1.64554450e-01 -1.19040517e-16  3.55926567e-15  9.86368001e-01]]]


In [5]:
psi0_MPS_compr = compress(psi0, L, 10)

remaining in compress:  [[-1.]]


In [6]:
print("total size of psi_MPS_ex =", sum([M.size for M in psi0_MPS_ex]))
print("total size of psi_MPS_compr =", sum([M.size for M in psi0_MPS_compr]))

total size of psi_MPS_ex = 43688
total size of psi_MPS_compr = 1688


In [7]:
 def overlap(mps_bra, mps_ket):
    L = len(mps_bra)
    assert L == len(mps_ket)
    contr = np.ones((1,1)) # has indices (alpha_n*, alpha_n), which we label vR* vR
    for n in range(L):
        M_ket = mps_ket[n]  # has indices (alpha_n, j_n, alpha_{n+1}) -> vL j vR
        contr = np.tensordot(contr, M_ket , axes=(1, 0)) # vR* [vR], [vL] j vR contract indices in []
        # now contr has indices vR* j vR
        M_bra = mps_bra[n].conj()  # vL* j* vR*
        contr = np.tensordot(M_bra, contr, axes=([0, 1], [0, 1])) # [vL*] [j*] vR*, [vR*] [j] vR
    assert contr.shape == (1, 1)
    return contr.item()
        

In [8]:
print(overlap(psi0_MPS_ex, psi0_MPS_ex))

1.0000000000000007


In [9]:
print(overlap(psi0_MPS_ex, psi0_MPS_compr))
print("Still very good overlap, given that we have compressed the state quite much!")

0.9999999999999889
Still very good overlap, given that we have compressed the state quite much!


In [10]:
M_up = np.zeros((1, 2, 1))
M_up[0, 0, 0] = 1.
MPS_all_up = [M_up.copy() for i in range(L)]

In [11]:
print(overlap(psi0_MPS_ex, MPS_all_up))

0.8146943347914248
